# Load Packages

In [1]:
import functions as f
import pandas as pd
from tqdm import tqdm
from joblib import Parallel, delayed
import numpy as np

/Users/victorperso/Documents/EUR/Aegon Marketing Case/baqm-seminar-16/.conda/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Load Data

In [2]:
df = pd.read_csv("../data/prepped_data.csv", low_memory=False, index_col=0).drop_duplicates()

# Run Double ML

In [3]:
first_stage_1, first_stage_2, double_mls, splits = f.global_run(df, splits=1, cols_to_drop_manual=['last_type'], iters=1, log=False, intermediary_scores=False)

Running Split 1...
Done!!


/Users/victorperso/Documents/EUR/Aegon Marketing Case/baqm-seminar-16/.conda/lib/python3.8/site-packages/doubleml/_utils_checks.py:204: UserWarning: Propensity predictions from learner Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression(max_iter=1000))]) for ml_m are close to zero or one (eps=1e-12).
  warnings.warn(f'Propensity predictions from learner {str(learner)} for'


In [5]:
for k, v in double_mls.items():
    display(v.summary)
    # v.sensitivity_analysis(cf_y=0.00898, cf_d=0.14166, rho=0.4833)
    v.sensitivity_analysis()
    print(v.sensitivity_summary)
    # v.sensitivity_plot()

,coef,std err,t,P>|t|,2.5 %,97.5 %
welcome_discount,0.089725,0.008096,11.083034,1.516452e-28,0.073858,0.105592


================== Sensitivity Analysis ==================

------------------ Scenario          ------------------
Significance Level: level=0.95
Sensitivity parameters: cf_y=0.03; cf_d=0.03, rho=1.0

------------------ Bounds with CI    ------------------
                  CI lower  theta lower     theta  theta upper  CI upper
welcome_discount  0.049189     0.063155  0.089725     0.116295  0.130669

------------------ Robustness Values ------------------
                  H_0    RV (%)   RVa (%)
welcome_discount  0.0  9.770818  7.868049


In [5]:
# for k_, v_ in double_mls.items():

#     print(k_)
#     display(v_.summary)
#     features = [col for col in splits[k_].columns if col not in ['welcome_discount', 'churn']]
#     benchmark_sensitivities = {}

#     def process_feature(feature):
#         return feature, v_.sensitivity_benchmark(benchmarking_set=[feature])

#     results = Parallel(n_jobs=-1)(delayed(process_feature)(feature) for feature in features)

#     for feature, result in results:
#         benchmark_sensitivities[feature] = result

#     cf_y_lst = []
#     cf_d_lst = []
#     names = []
#     rhos = []

#     for k, v in benchmark_sensitivities.items():
#         cf_y_lst.append(v.loc["welcome_discount", "cf_y"])
#         cf_d_lst.append(v.loc["welcome_discount", "cf_d"])
#         rhos.append(v.loc["welcome_discount", "rho"])
#         names.append(k)

#     benchmark_dict = {
#         "cf_y" : cf_y_lst,
#         "cf_d" : cf_d_lst,
#         "name" : names
#     }

#     v_.sensitivity_analysis(cf_y=0.04, cf_d=0.03)
#     v_.sensitivity_plot(benchmarks=benchmark_dict)

# print(f"Max cf_y: {np.max(cf_y_lst)}")
# print(f"Max cf_d: {np.max(cf_d_lst)}")
# print(f"Max rho: {np.max([np.abs(rho) for rho in rhos if np.abs(rho) != 1.0])}")